<a href="https://colab.research.google.com/github/msironvalle/WebSemanticaMTI2025/blob/main/Tarea_2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer Set de datos

In [ ]:
!pip install rdflib
!pip install sparqlwrapper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 12.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import rdflib
from rdflib import Graph

In [ ]:
# =====================================
# 4) Definición de la ruta a tu archivo RDF en Drive
# =====================================
rdf_file_path = "/content/drive/My Drive/output_PasosFronterizos_7.rdf"

In [ ]:
g = Graph()

# Si tu archivo está en formato Turtle (extensión .ttl o con prefijos en Turtle)
# especifica format="turtle". Si fuera RDF/XML, usar format="xml".
g.parse(rdf_file_path, format="turtle")

# =====================================
# 6) Ejemplo de consultas SPARQL
# =====================================
# 6.1) Consulta para obtener el paso fronterizo con la mayor altura
query_mayor_altura = """
PREFIX ex: <http://example.org/pasos_fronterizos#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?paso ?nombre ?altura
WHERE {
  ?paso a ex:PasoFronterizo ;
        ex:nombre ?nombre ;
        ex:altura ?altura .
}
ORDER BY DESC(?altura)
LIMIT 1
"""

# 6.2) Consulta para obtener el paso fronterizo más al sur (lat más pequeña)
query_mas_al_sur = """
PREFIX ex: <http://example.org/pasos_fronterizos#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?paso ?nombre ?lat
WHERE {
  ?paso a ex:PasoFronterizo ;
        ex:nombre ?nombre ;
        geo:lat ?lat .
}
ORDER BY ASC(?lat)
LIMIT 1
"""

# =====================================
# 7) Ejecutar consultas y mostrar resultados
# =====================================
# Consulta 1: Paso fronterizo de mayor altura
print("==== Paso fronterizo de mayor altura ====")
for row in g.query(query_mayor_altura):
    print(f"URI: {row.paso}, Nombre: {row.nombre}, Altura: {row.altura}")

print()

# Consulta 2: Paso fronterizo más al sur
print("==== Paso fronterizo más al sur ====")
for row in g.query(query_mas_al_sur):
    print(f"URI: {row.paso}, Nombre: {row.nombre}, Latitud: {row.lat}")

==== Paso fronterizo de mayor altura ====
URI: http://example.org/pasos_fronterizos#PasoFronterizo_15, Nombre: Coyhaique, Altura: 795

==== Paso fronterizo más al sur ====
URI: http://example.org/pasos_fronterizos#PasoFronterizo_31, Nombre: Bellavista, Latitud: -54


## Consulta Federada

In [ ]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

# ==============================================
# 3. Cargar el RDF local en un grafo
#    Ajusta la ruta si fuera distinto el archivo
# ==============================================
local_rdf_path = "/content/drive/My Drive/output_PasosFronterizos_7.rdf"

g_local = Graph()
# Tu RDF está en Turtle, así que usamos 'turtle'
g_local.parse(local_rdf_path, format="turtle")

# ==============================================
# 4. Consulta local para obtener datos de "pasos fronterizos"
# ==============================================
query_local = """
PREFIX ex: <http://example.org/pasos_fronterizos#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?paso ?nombre ?altura ?region ?unidad ?lat ?long
WHERE {
  ?paso a ex:PasoFronterizo ;
        ex:nombre ?nombre ;
        ex:altura ?altura ;
        ex:region ?region ;
        ex:unidad ?unidad ;
        geo:lat ?lat ;
        geo:long ?long .
}
"""

results_local = g_local.query(query_local)

print("=== PASOS FRONTERIZOS EN RDF LOCAL ===\n")
nombres_locales = []
for row in results_local:
    paso_uri = str(row.paso)
    nombre = str(row.nombre)
    altura = str(row.altura)
    region = str(row.region)
    unidad = str(row.unidad)
    lat = str(row.lat)
    long_ = str(row.long)

    print(f"URI: {paso_uri}")
    print(f"  Nombre: {nombre}")
    print(f"  Altura: {altura} {unidad}")
    print(f"  Región: {region}")
    print(f"  Coordenadas: ({lat}, {long_})")
    print("-"*50)

    nombres_locales.append(nombre)

# ==============================================
# 5. Consulta a Wikidata: todos los "border crossings" en Chile
#    (esto es una segunda fuente de datos)
# ==============================================
# wdt:P31/wdt:P279* wd:Q28769114 => instancia o subclase de "border crossing"
# wdt:P17 wd:Q298               => el país es Chile
# El SERVICE wikibase:label permite traer etiquetas en español.
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

query_wikidata = """
SELECT ?paso ?pasoLabel
WHERE {
  ?paso wdt:P31/wdt:P279* wd:Q28769114 ;
        wdt:P17 wd:Q298 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "es,en". }
}
ORDER BY ?pasoLabel
"""

sparql.setQuery(query_wikidata)
res_wikidata = sparql.query().convert()

# Transformamos resultados en una lista
pasos_wikidata = []
for b in res_wikidata["results"]["bindings"]:
    qitem = b["paso"]["value"]
    label = b.get("pasoLabel", {}).get("value", "")
    pasos_wikidata.append((qitem, label))

print("\n=== PASOS INTERNACIONALES DE CHILE SEGÚN WIKIDATA ===\n")
for q, lbl in pasos_wikidata:
    print(f"- {lbl} -> {q}")

# ==============================================
# 6. (Opcional) Ver coincidencias de nombres
#    entre el RDF local y Wikidata
# ==============================================
print("\n=== COINCIDENCIAS DE NOMBRES (Local vs Wikidata) ===\n")

set_local = set(nombres_locales)
for q, lbl in pasos_wikidata:
    # Chequeamos si el label WD coincide con un nombre local exacto
    if lbl in set_local:
        print(f"Coincide: {lbl}  (Q-item: {q})")

=== PASOS FRONTERIZOS EN RDF LOCAL ===

URI: http://example.org/pasos_fronterizos#PasoFronterizo_1
  Nombre: Salar de Ollague
  Altura: 4 Metros
  Región: Antofagasta
  Coordenadas: (-21.2230556, -68.24944444)
--------------------------------------------------
URI: http://example.org/pasos_fronterizos#PasoFronterizo_2
  Nombre: Portezuelo de Cajón
  Altura: 4 Metros
  Región: Antofagasta
  Coordenadas: (-22.9161111, -67.79944444)
--------------------------------------------------
URI: http://example.org/pasos_fronterizos#PasoFronterizo_3
  Nombre: San Pedro de Atacama
  Altura: 3 Metros
  Región: Antofagasta
  Coordenadas: (-22.9105556, -68.19361111)
--------------------------------------------------
URI: http://example.org/pasos_fronterizos#PasoFronterizo_4
  Nombre: Jama
  Altura: 4 Metros
  Región: Antofagasta
  Coordenadas: (-23.2369444, -67.02277778)
--------------------------------------------------
URI: http://example.org/pasos_fronterizos#PasoFronterizo_5
  Nombre: Sico
  Altur

# Segundo Set de Datos

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL para obtener los códigos postales...")

# Ejecutar la consulta
results = g.query(sparql_query)

# Contar colegios por área postal (2 primeras letras del código postal)
area_counts = collections.Counter()

for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer las primeras 2 letras
    area_counts[area_code] += 1

# Mostrar resultados
print("\n📌 Cantidad de colegios por área postal:")
for area, count in sorted(area_counts.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL para obtener los códigos postales...

📌 Cantidad de colegios por área postal:
Área Postal: DH, Cantidad de Escuelas: 116
Área Postal: DL, Cantidad de Escuelas: 104
Área Postal: SR, Cantidad de Escuelas: 30
Área Postal: TS, Cantidad de Escuelas: 16
Área Postal: NE, Cantidad de Escuelas: 1


Federada

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections
from SPARQLWrapper import SPARQLWrapper, JSON

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL para obtener los códigos postales...")

# Ejecutar la consulta
results = g.query(sparql_query)

# Extraer las primeras 2 letras de cada código postal (área postal)
area_codes = set()
for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer las primeras 2 letras
    area_codes.add(area_code)

print(f"\n📌 Áreas postales detectadas en los datos locales: {', '.join(area_codes)}")

# --- CONSULTA FEDERADA A WIKIDATA ---
print("\n🔹 Consultando colegios públicos en Wikidata según área postal...")

# Configurar la consulta SPARQL en Wikidata
wikidata_sparql = """
SELECT ?school ?schoolLabel ?postcode WHERE {
  ?school wdt:P31 wd:Q3914;  # Instancia de "escuela"
          wdt:P17 wd:Q145;   # Ubicada en Reino Unido
          wdt:P281 ?postcode. # Código postal

  # Filtrar por las áreas postales detectadas en el RDF local
  FILTER (""" + " || ".join([f"STRSTARTS(?postcode, \"{code}\")" for code in area_codes]) + """ )

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

# Conectar con Wikidata y ejecutar la consulta
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wikidata_sparql)
sparql.setReturnFormat(JSON)
results_wikidata = sparql.query().convert()

# Mostrar resultados obtenidos de Wikidata
print("\n📌 Colegios públicos en Reino Unido según área postal (desde Wikidata):")
for result in results_wikidata["results"]["bindings"]:
    school_name = result["schoolLabel"]["value"]
    school_postcode = result["postcode"]["value"]
    print(f"🏫 {school_name} - Código Postal: {school_postcode}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL para obtener los códigos postales...

📌 Áreas postales detectadas en los datos locales: NE, DH, SR, TS, DL

🔹 Consultando colegios públicos en Wikidata según área postal...

📌 Colegios públicos en Reino Unido según área postal (desde Wikidata):
🏫 Argyle House School - Código Postal: SR2 7LA
🏫 Barnard Castle School - Código Postal: DL12 8UN
🏫 Chorister School - Código Postal: DH1 3EL
🏫 Eastbourne Church of England Academy - Código Postal: DL1 1LL
🏫 The Grangefield Academy - Código Postal: TS18 4LE
🏫 Grindon Hall Christian School - Código Postal: SR4 8PG
🏫 St John's School & Sixth Form College - A Catholic Academy - Código Postal: DL14 6JT
🏫 St Joseph's R.C. Middle School - Código Postal: NE46 2DD
🏫 St Leonard's Catholic School, Durham - Código Postal: DH1 4NG
🏫 St Wilfrid's 

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo RDF cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios desde RDF
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...")

# Ejecutar la consulta en el RDF local
results = g.query(sparql_query)

# Contar colegios por área postal (primeras 2 letras del código postal)
area_counts_local = collections.Counter()

for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer primeras 2 letras
    area_counts_local[area_code] += 1

print("\n📌 Cantidad de escuelas por área postal en RDF local:")
for area, count in sorted(area_counts_local.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- CONSULTA FEDERADA A WIKIDATA ---
print("\n🔹 Consultando cantidad de escuelas en Wikidata por área postal...")

# Definir la consulta SPARQL en Wikidata
wikidata_sparql = """
SELECT ?postcode (COUNT(?school) AS ?count) WHERE {
  ?school wdt:P31 wd:Q3914;  # Instancia de "escuela"
          wdt:P17 wd:Q145;   # Ubicada en Reino Unido
          wdt:P281 ?postcode. # Código postal
}
GROUP BY ?postcode
"""

# Conectar con Wikidata y ejecutar la consulta
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wikidata_sparql)
sparql.setReturnFormat(JSON)
results_wikidata = sparql.query().convert()

# Contar colegios por área postal en Wikidata
area_counts_wikidata = collections.Counter()

for result in results_wikidata["results"]["bindings"]:
    postcode = result["postcode"]["value"]  # Obtener código postal
    area_code = postcode[:2]  # Extraer primeras 2 letras
    count = int(result["count"]["value"])  # Convertir a entero
    area_counts_wikidata[area_code] += count  # Contar colegios por área

print("\n📌 Cantidad de escuelas por área postal en Wikidata:")
for area, count in sorted(area_counts_wikidata.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- COMPARAR DATOS ENTRE RDF LOCAL Y WIKIDATA ---
print("\n🔹 Comparando datos entre RDF local y Wikidata...")

# Crear un DataFrame para visualizar la comparación
data = []
for area in set(area_counts_local.keys()).union(area_counts_wikidata.keys()):
    local_count = area_counts_local.get(area, 0)
    wikidata_count = area_counts_wikidata.get(area, 0)
    data.append([area, local_count, wikidata_count])

# Convertir a DataFrame
df_comparison = pd.DataFrame(data, columns=["Área Postal", "Escuelas en RDF Local", "Escuelas en Wikidata"])
df_comparison = df_comparison.sort_values(by="Escuelas en Wikidata", ascending=False)

# Mostrar la tabla
import ace_tools as tools
tools.display_dataframe_to_user(name="Comparación de Escuelas por Área Postal", dataframe=df_comparison)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo RDF cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...

📌 Cantidad de escuelas por área postal en RDF local:
Área Postal: DH, Cantidad de Escuelas: 116
Área Postal: DL, Cantidad de Escuelas: 104
Área Postal: SR, Cantidad de Escuelas: 30
Área Postal: TS, Cantidad de Escuelas: 16
Área Postal: NE, Cantidad de Escuelas: 1

🔹 Consultando cantidad de escuelas en Wikidata por área postal...

📌 Cantidad de escuelas por área postal en Wikidata:
Área Postal: RG, Cantidad de Escuelas: 28
Área Postal: GU, Cantidad de Escuelas: 27
Área Postal: RH, Cantidad de Escuelas: 23
Área Postal: OX, Cantidad de Escuelas: 20
Área Postal: SW, Cantidad de Escuelas: 18
Área Postal: KT, Cantidad de Escuelas: 18
Área Postal: TN, Cantidad de Escuelas: 17
Área Postal: SE, Cantidad de Escuelas: 16
Área Post

ModuleNotFoundError: No module named 'ace_tools'

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from IPython.display import display  # <- 🔹 Corregido: Para mostrar la tabla en Google Colab

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo RDF cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios desde RDF
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...")

# Ejecutar la consulta en el RDF local
results = g.query(sparql_query)

# Contar colegios por área postal (primeras 2 letras del código postal)
area_counts_local = collections.Counter()

for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer primeras 2 letras
    area_counts_local[area_code] += 1

print("\n📌 Cantidad de escuelas por área postal en RDF local:")
for area, count in sorted(area_counts_local.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- CONSULTA FEDERADA A WIKIDATA ---
print("\n🔹 Consultando cantidad de escuelas en Wikidata por área postal...")

# Definir la consulta SPARQL en Wikidata
wikidata_sparql = """
SELECT ?postcode (COUNT(?school) AS ?count) WHERE {
  ?school wdt:P31 wd:Q3914;  # Instancia de "escuela"
          wdt:P17 wd:Q145;   # Ubicada en Reino Unido
          wdt:P281 ?postcode. # Código postal
}
GROUP BY ?postcode
"""

# Conectar con Wikidata y ejecutar la consulta
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wikidata_sparql)
sparql.setReturnFormat(JSON)
results_wikidata = sparql.query().convert()

# Contar colegios por área postal en Wikidata
area_counts_wikidata = collections.Counter()

for result in results_wikidata["results"]["bindings"]:
    postcode = result["postcode"]["value"]  # Obtener código postal
    area_code = postcode[:2]  # Extraer primeras 2 letras
    count = int(result["count"]["value"])  # Convertir a entero
    area_counts_wikidata[area_code] += count  # Contar colegios por área

print("\n📌 Cantidad de escuelas por área postal en Wikidata:")
for area, count in sorted(area_counts_wikidata.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- COMPARAR DATOS ENTRE RDF LOCAL Y WIKIDATA ---
print("\n🔹 Comparando datos entre RDF local y Wikidata...")

# Crear un DataFrame para visualizar la comparación
data = []
for area in set(area_counts_local.keys()).union(area_counts_wikidata.keys()):
    local_count = area_counts_local.get(area, 0)
    wikidata_count = area_counts_wikidata.get(area, 0)
    data.append([area, local_count, wikidata_count])

# Convertir a DataFrame
df_comparison = pd.DataFrame(data, columns=["Área Postal", "Escuelas en RDF Local", "Escuelas en Wikidata"])
df_comparison = df_comparison.sort_values(by="Escuelas en Wikidata", ascending=False)

# 🔹 Mostrar la tabla con Pandas en Google Colab
print("\n📌 Comparación de Escuelas por Área Postal:")
display(df_comparison)  # <- 🔹 CORREGIDO: Mostrar la tabla en Google Colab


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo RDF cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...

📌 Cantidad de escuelas por área postal en RDF local:
Área Postal: DH, Cantidad de Escuelas: 116
Área Postal: DL, Cantidad de Escuelas: 104
Área Postal: SR, Cantidad de Escuelas: 30
Área Postal: TS, Cantidad de Escuelas: 16
Área Postal: NE, Cantidad de Escuelas: 1

🔹 Consultando cantidad de escuelas en Wikidata por área postal...

📌 Cantidad de escuelas por área postal en Wikidata:
Área Postal: RG, Cantidad de Escuelas: 28
Área Postal: GU, Cantidad de Escuelas: 27
Área Postal: RH, Cantidad de Escuelas: 23
Área Postal: OX, Cantidad de Escuelas: 20
Área Postal: SW, Cantidad de Escuelas: 18
Área Postal: KT, Cantidad de Escuelas: 18
Área Postal: TN, Cantidad de Escuelas: 17
Área Postal: SE, Cantidad de Escuelas: 16
Área Post

,Área Postal,Escuelas en RDF Local,Escuelas en Wikidata
115,RG,0,28
27,GU,0,27
122,RH,0,23
50,OX,0,20
25,SW,0,18
...,...,...,...
105,B5,0,1
52,B6,0,1
67,BR,0,1
97,BT,0,1


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from IPython.display import display  # Para mostrar tablas en Google Colab

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo RDF cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios desde RDF
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...")

# Ejecutar la consulta en el RDF local
results = g.query(sparql_query)

# Contar colegios por área postal (primeras 2 letras del código postal)
area_counts_local = collections.Counter()

for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer primeras 2 letras
    area_counts_local[area_code] += 1

print("\n📌 Cantidad de escuelas por área postal en RDF local:")
for area, count in sorted(area_counts_local.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- CONSULTA FEDERADA A WIKIDATA ---
print("\n🔹 Consultando cantidad de escuelas en Wikidata por área postal...")

# Definir la consulta SPARQL en Wikidata
wikidata_sparql = """
SELECT ?postcode (COUNT(?school) AS ?count) WHERE {
  ?school wdt:P31 wd:Q3914;  # Instancia de "escuela"
          wdt:P17 wd:Q145;   # Ubicada en Reino Unido
          wdt:P281 ?postcode. # Código postal
}
GROUP BY ?postcode
"""

# Conectar con Wikidata y ejecutar la consulta
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wikidata_sparql)
sparql.setReturnFormat(JSON)
results_wikidata = sparql.query().convert()

# Contar colegios por área postal en Wikidata
area_counts_wikidata = collections.Counter()

for result in results_wikidata["results"]["bindings"]:
    postcode = result["postcode"]["value"]  # Obtener código postal
    area_code = postcode[:2]  # Extraer primeras 2 letras
    count = int(result["count"]["value"])  # Convertir a entero
    area_counts_wikidata[area_code] += count  # Contar colegios por área

print("\n📌 Cantidad de escuelas por área postal en Wikidata:")
for area, count in sorted(area_counts_wikidata.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- COMPARAR DATOS ENTRE RDF LOCAL Y WIKIDATA ---
print("\n🔹 Comparando datos entre RDF local y Wikidata...")

# Crear DataFrame para comparar TODAS las áreas postales
data_general = []
for area in set(area_counts_local.keys()).union(area_counts_wikidata.keys()):
    local_count = area_counts_local.get(area, 0)
    wikidata_count = area_counts_wikidata.get(area, 0)
    data_general.append([area, local_count, wikidata_count])

# Convertir a DataFrame
df_general = pd.DataFrame(data_general, columns=["Área Postal", "Escuelas en RDF Local", "Escuelas en Wikidata"])
df_general = df_general.sort_values(by="Escuelas en Wikidata", ascending=False)

# Crear DataFrame solo con áreas postales que existen en el RDF local
data_local_only = []
for area in area_counts_local.keys():  # Solo las áreas postales que están en el RDF
    local_count = area_counts_local[area]
    wikidata_count = area_counts_wikidata.get(area, 0)  # Puede que Wikidata no tenga datos
    data_local_only.append([area, local_count, wikidata_count])

df_local_only = pd.DataFrame(data_local_only, columns=["Área Postal", "Escuelas en RDF Local", "Escuelas en Wikidata"])
df_local_only = df_local_only.sort_values(by="Escuelas en RDF Local", ascending=False)

# --- MOSTRAR RESULTADOS ---
print("\n📌 Comparación General de Escuelas por Área Postal:")
display(df_general)  # Mostrar tabla con TODAS las áreas postales

print("\n📌 Comparación SOLO para áreas postales en RDF local:")
display(df_local_only)  # Mostrar tabla solo con áreas que existen en RDF local


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo RDF cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...

📌 Cantidad de escuelas por área postal en RDF local:
Área Postal: DH, Cantidad de Escuelas: 116
Área Postal: DL, Cantidad de Escuelas: 104
Área Postal: SR, Cantidad de Escuelas: 30
Área Postal: TS, Cantidad de Escuelas: 16
Área Postal: NE, Cantidad de Escuelas: 1

🔹 Consultando cantidad de escuelas en Wikidata por área postal...

📌 Cantidad de escuelas por área postal en Wikidata:
Área Postal: RG, Cantidad de Escuelas: 28
Área Postal: GU, Cantidad de Escuelas: 27
Área Postal: RH, Cantidad de Escuelas: 23
Área Postal: OX, Cantidad de Escuelas: 20
Área Postal: SW, Cantidad de Escuelas: 18
Área Postal: KT, Cantidad de Escuelas: 18
Área Postal: TN, Cantidad de Escuelas: 17
Área Postal: SE, Cantidad de Escuelas: 16
Área Post

,Área Postal,Escuelas en RDF Local,Escuelas en Wikidata
115,RG,0,28
27,GU,0,27
122,RH,0,23
50,OX,0,20
25,SW,0,18
...,...,...,...
105,B5,0,1
52,B6,0,1
67,BR,0,1
97,BT,0,1



📌 Comparación SOLO para áreas postales en RDF local:


,Área Postal,Escuelas en RDF Local,Escuelas en Wikidata
2,DH,116,4
0,DL,104,8
1,SR,30,3
3,TS,16,5
4,NE,1,14


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías necesarias
from rdflib import Graph, Namespace
import collections
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from IPython.display import display  # Para mostrar tablas en Google Colab

# Ruta del archivo RDF en Google Drive
rdf_file_path = "/content/drive/My Drive/output_SchoolsDurham.rdf"

# Cargar el archivo RDF en memoria
g = Graph()
g.parse(rdf_file_path, format="turtle")  # Usamos formato Turtle

print(f"✅ Archivo RDF cargado correctamente. Número de triples: {len(g)}")

# Definir el prefijo correcto basado en la inspección previa
ex = Namespace("http://example.org/schools#")

# Consulta SPARQL para obtener los códigos postales de los colegios desde RDF
sparql_query = """
PREFIX ex: <http://example.org/schools#>
SELECT ?postcode
WHERE {
  ?school ex:postcode ?postcode .
}
"""

print("\n🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...")

# Ejecutar la consulta en el RDF local
results = g.query(sparql_query)

# Contar colegios por área postal (primeras 2 letras del código postal)
area_counts_local = collections.Counter()

for row in results:
    postcode = str(row.postcode)  # Convertir a string
    area_code = postcode[:2]  # Extraer primeras 2 letras
    area_counts_local[area_code] += 1

print("\n📌 Cantidad de escuelas por área postal en RDF local:")
for area, count in sorted(area_counts_local.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Escuelas: {count}")

# --- CONSULTA FEDERADA A WIKIDATA ---
print("\n🔹 Consultando cantidad de instituciones educativas en Wikidata por área postal...")

# Definir la consulta SPARQL en Wikidata (Incluye múltiples categorías de instituciones educativas)
wikidata_sparql = """
SELECT ?postcode (COUNT(?institution) AS ?count) WHERE {
  ?institution wdt:P31 ?type;  # Instancia de instituciones educativas
               wdt:P17 wd:Q145;   # Ubicada en Reino Unido
               wdt:P281 ?postcode. # Código postal
  VALUES ?type {
    wd:Q3914    # Schools
    wd:Q1930187 # Academies
    wd:Q8250    # Colegios secundarios
    wd:Q9842    # Colegios primarios
    wd:Q13218630 # Colegios especializados
    wd:Q178706  # Escuelas independientes
  }
}
GROUP BY ?postcode
"""

# Conectar con Wikidata y ejecutar la consulta
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wikidata_sparql)
sparql.setReturnFormat(JSON)
results_wikidata = sparql.query().convert()

# Contar instituciones educativas por área postal en Wikidata
area_counts_wikidata = collections.Counter()

for result in results_wikidata["results"]["bindings"]:
    postcode = result["postcode"]["value"]  # Obtener código postal
    area_code = postcode[:2]  # Extraer primeras 2 letras
    count = int(result["count"]["value"])  # Convertir a entero
    area_counts_wikidata[area_code] += count  # Contar instituciones por área

print("\n📌 Cantidad de instituciones educativas por área postal en Wikidata:")
for area, count in sorted(area_counts_wikidata.items(), key=lambda x: -x[1]):
    print(f"Área Postal: {area}, Cantidad de Instituciones: {count}")

# --- COMPARAR DATOS ENTRE RDF LOCAL Y WIKIDATA ---
print("\n🔹 Comparando datos entre RDF local y Wikidata...")

# Crear DataFrame para comparar TODAS las áreas postales
data_general = []
for area in set(area_counts_local.keys()).union(area_counts_wikidata.keys()):
    local_count = area_counts_local.get(area, 0)
    wikidata_count = area_counts_wikidata.get(area, 0)
    data_general.append([area, local_count, wikidata_count])

# Convertir a DataFrame
df_general = pd.DataFrame(data_general, columns=["Área Postal", "Escuelas en RDF Local", "Instituciones en Wikidata"])
df_general = df_general.sort_values(by="Instituciones en Wikidata", ascending=False)

# Crear DataFrame solo con áreas postales que existen en el RDF local
data_local_only = []
for area in area_counts_local.keys():  # Solo las áreas postales que están en el RDF
    local_count = area_counts_local[area]
    wikidata_count = area_counts_wikidata.get(area, 0)  # Puede que Wikidata no tenga datos
    data_local_only.append([area, local_count, wikidata_count])

df_local_only = pd.DataFrame(data_local_only, columns=["Área Postal", "Escuelas en RDF Local", "Instituciones en Wikidata"])
df_local_only = df_local_only.sort_values(by="Escuelas en RDF Local", ascending=False)

# --- MOSTRAR RESULTADOS ---
print("\n📌 Comparación General de Instituciones Educativas por Área Postal:")
display(df_general)  # Mostrar tabla con TODAS las áreas postales

print("\n📌 Comparación SOLO para áreas postales en RDF local:")
display(df_local_only)  # Mostrar tabla solo con áreas que existen en RDF local


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo RDF cargado correctamente. Número de triples: 2671

🔹 Ejecutando consulta SPARQL en RDF local para obtener los códigos postales...

📌 Cantidad de escuelas por área postal en RDF local:
Área Postal: DH, Cantidad de Escuelas: 116
Área Postal: DL, Cantidad de Escuelas: 104
Área Postal: SR, Cantidad de Escuelas: 30
Área Postal: TS, Cantidad de Escuelas: 16
Área Postal: NE, Cantidad de Escuelas: 1

🔹 Consultando cantidad de instituciones educativas en Wikidata por área postal...

📌 Cantidad de instituciones educativas por área postal en Wikidata:
Área Postal: NG, Cantidad de Instituciones: 390
Área Postal: NE, Cantidad de Instituciones: 371
Área Postal: PE, Cantidad de Instituciones: 338
Área Postal: LE, Cantidad de Instituciones: 312
Área Postal: CF, Cantidad de Instituciones: 309
Área Postal: DE, Cantidad de Instituciones: 308
Área Postal: BS, Cantidad

,Área Postal,Escuelas en RDF Local,Instituciones en Wikidata
59,NG,0,390
9,NE,1,371
70,PE,0,338
129,LE,0,312
68,CF,0,309
...,...,...,...
167,KA,0,1
38,PH,0,1
116,G3,0,1
150,FK,0,1



📌 Comparación SOLO para áreas postales en RDF local:


,Área Postal,Escuelas en RDF Local,Instituciones en Wikidata
2,DH,116,123
0,DL,104,169
1,SR,30,76
3,TS,16,203
4,NE,1,371
